In [1]:
import json
import os
import numpy as np

# --- 1. 读取 JSON 文件 ---
file_path = "uuv_visualizations_sparse_gp_submit_global_3d2/simulation_results.json"

with open(file_path, "r") as f:
    data = json.load(f)


# --- 2. 函数：调整坐标 (xy缩放, z偏移, 可选 z缩放) ---
def adjust_coords(coords, scale_xy=1.5, offset_z=-30, scale_z=1.0):
    """将 x、y 坐标等比例放大，z 坐标加上偏移量并可选缩放"""
    coords = np.array(coords, dtype=np.float64)  # 确保为浮点数，避免类型冲突
    if coords.ndim == 1:  # 处理单个坐标
        coords[0:2] *= scale_xy
        coords[2] = coords[2] * scale_z + offset_z
    else:  # 处理多个坐标
        coords[:, 0:2] *= scale_xy
        coords[:, 2] = coords[:, 2] * scale_z + offset_z
    return coords.tolist()


# --- 3. 修改所有坐标 ---
# 设置缩放参数
SCALE_XY = 2    # x, y 的缩放比例
OFFSET_Z = -30    # z 的偏移量
SCALE_Z = 1.0     # z 的缩放比例 (1.0 = 不缩放)

# 1. UUV 初始位置
data["uuv_initial_positions"] = adjust_coords(data["uuv_initial_positions"], scale_xy=SCALE_XY, offset_z=OFFSET_Z, scale_z=SCALE_Z)

# 2. 每轮拍卖后 UUV 的目标位置
for step in data["uuv_targets_per_step"]:
    for key, val in step.items():
        step[key] = adjust_coords(val, scale_xy=SCALE_XY, offset_z=OFFSET_Z, scale_z=SCALE_Z)

# 3. GMM 的均值位置
data["gmm_means"] = adjust_coords(data["gmm_means"], scale_xy=SCALE_XY, offset_z=OFFSET_Z, scale_z=SCALE_Z)

# 4. 目标位置
data["target_position"] = adjust_coords(data["target_position"], scale_xy=SCALE_XY, offset_z=OFFSET_Z, scale_z=SCALE_Z)


# --- 4. 保存修改后的 JSON 文件 ---
output_path = "uuv_visualizations_sparse_gp_submit_global_3d2/simulation_results_modified.json"
with open(output_path, "w") as f:
    json.dump(data, f, indent=4)

print("✅ 坐标已调整：x、y 等比例放大，z 坐标偏移，并保存至:")
print(output_path)


✅ 坐标已调整：x、y 等比例放大，z 坐标偏移，并保存至:
uuv_visualizations_sparse_gp_submit_global_3d2/simulation_results_modified.json


In [2]:
import holoocean
import numpy as np
import json
import os
import time
import csv

# --- 1. 加载 simulation_results_modified.json ---
file_path = "uuv_visualizations_sparse_gp_submit_global_3d/simulation_results_modified.json"

with open(file_path, "r") as f:
    data = json.load(f)

uuv_initial_positions = np.array(data["uuv_initial_positions"])
uuv_targets_per_step = data["uuv_targets_per_step"]
gmm_means = np.array(data["gmm_means"])
target_position = np.array(data["target_position"])

# --- 2. HoloOcean 配置 (3个UUV, 1个TorpedoAUV) ---
config = {
    "name": "ThreeUUVSimulation",
    "world": "SimpleUnderwater",
    "package_name": "Ocean",
    "main_agent": "auv0",
    "agents": [
        {
            "agent_name": f"auv{i}",
            "agent_type": "HoveringAUV",
            "sensors": [
                {
                    "sensor_type": "GPSSensor",
                    "socket": "COM",
                    "configuration": {
                        "Sigma": 0.0,
                        "Depth": 100
                    }
                }
            ],
            "control_scheme": 1,  # PD Control Scheme
            "location": uuv_initial_positions[i].tolist()
        } for i in range(3)
    ] + [
        {
            "agent_name": "torpedo",
            "agent_type": "TorpedoAUV",
            "sensors": [],
            "control_scheme": 0,  # 无需控制，设为0
            "location": target_position.tolist()
        }
    ]
}

# --- 3. 模拟参数 ---
max_steps = 5000
arrival_tolerance = 0.1  # 小于此距离即为到达目标
save_folder = "uuv_simulation_output"
os.makedirs(save_folder, exist_ok=True)

# --- 4. 数据记录 ---
positions = {f"auv{i}": [] for i in range(3)}
arrival_times = {f"auv{i}": [] for i in range(3)}

# 记录每个UUV的上一次目标和位置
last_targets = {f"auv{i}": None for i in range(3)}
previous_positions = {f"auv{i}": None for i in range(3)}  # 记录每个 UUV 上一次的位置

# 尾迹颜色设置
trail_colors = {
    "auv0": [255, 0, 0],   # 红色
    "auv1": [0, 255, 0],   # 绿色
    "auv2": [0, 0, 255]    # 蓝色
}

# --- 5. 主程序 ---
with holoocean.make(scenario_cfg=config) as env:
    step_counter = 0
    round_idx = 0

    # 为每个 UUV 单独维护轮次索引
    uuv_round_indices = {f"auv{i}": 0 for i in range(3)}
    uuv_targets_done = {f"auv{i}": False for i in range(3)}

    # --- 🔥 可视化 TorpedoAUV ---
    # env.draw_point(target_position.tolist(), color=[255, 165, 0], thickness=100, lifetime=0)  # 橙色更醒目

    while step_counter <= max_steps:
        step_counter += 1

        # 为每个 UUV 发送控制指令
        for i in range(3):
            auv_name = f"auv{i}"
            current_round_idx = uuv_round_indices[auv_name]

            # 如果所有轮次完成，则跳过当前 UUV
            if current_round_idx >= len(uuv_targets_per_step):
                uuv_targets_done[auv_name] = True
                continue

            # 获取当前 UUV 的目标
            step_targets = uuv_targets_per_step[current_round_idx]
            target = step_targets.get(f"UUV_{i}", None)

            # 如果目标存在，发送控制指令
            if target is not None:
                last_targets[auv_name] = np.array(target)
                env.draw_point(target, thickness=10, lifetime=0)
                env.act(auv_name, target)

        # 推进仿真
        states = env.tick()

        # 获取当前位置并记录
        for i in range(3):
            auv_name = f"auv{i}"
            current_round_idx = uuv_round_indices[auv_name]

            # 如果当前 UUV 所有轮次已完成，跳过
            if uuv_targets_done[auv_name]:
                continue

            # 检查当前位置
            try:
                current_pos = states[auv_name]["GPSSensor"][0:3]
                positions[auv_name].append(current_pos)

                # --- 🟢 新增: 绘制 UUV 尾迹 (从 previous_positions 到 current_pos) ---
                if previous_positions[auv_name] is not None:
                    env.draw_line(
                        start=previous_positions[auv_name].tolist(),
                        end=current_pos.tolist(),
                        color=trail_colors[auv_name],
                        thickness=10.0,  # 尾迹线条粗细
                        lifetime=0        # 0 表示永久存在
                    )
                previous_positions[auv_name] = current_pos  # 更新上一次的位置

                # 判断是否到达目标
                target = last_targets[auv_name]
                if np.linalg.norm(current_pos - target) <= arrival_tolerance:
                    arrival_times[auv_name].append(step_counter)
                    print(f"✅ {auv_name} reached target {target} at step {step_counter}")

                    # 进入下一轮次
                    uuv_round_indices[auv_name] += 1
                    if uuv_round_indices[auv_name] >= len(uuv_targets_per_step):
                        uuv_targets_done[auv_name] = True

            except KeyError as e:
                print(f"❌ Error: {e}, skipping {auv_name}")

        # 如果所有 UUV 完成所有轮次，结束仿真
        if all(uuv_targets_done.values()):
            print("🏁 All UUVs completed all targets. Simulation ended.")
            # break

    print("\n✅ Simulation completed!")

    # # --- 6. 保存 UUV 轨迹到 CSV ---
    # for i in range(3):
    #     auv_name = f"auv{i}"
    #     trajectory = positions[auv_name]

    #     # 定义 CSV 文件名
    #     csv_file = os.path.join(save_folder, f"{auv_name}_trajectory.csv")

    #     # 写入 CSV 文件
    #     with open(csv_file, mode="w", newline="") as file:
    #         writer = csv.writer(file)
    #         writer.writerow(["step", "x", "y", "z"])  # 写入表头
    #         for step, pos in enumerate(trajectory):
    #             writer.writerow([step, pos[0], pos[1], pos[2]])  # 写入每一行数据

    #     print(f"✅ {auv_name} 的轨迹已保存到 {csv_file}")

✅ auv2 reached target [ 22.10526316   7.89473684 -15.26315789] at step 638
✅ auv2 reached target [ 22.10526316   7.89473684 -15.26315789] at step 639
✅ auv2 reached target [ 22.10526316   7.89473684 -15.26315789] at step 640
✅ auv0 reached target [  7.89473684  22.10526316 -15.26315789] at step 659
✅ auv0 reached target [  7.89473684  22.10526316 -15.26315789] at step 660
✅ auv0 reached target [  7.89473684  22.10526316 -15.26315789] at step 661
✅ auv1 reached target [ 23.68421053  23.68421053 -24.73684211] at step 773
✅ auv1 reached target [ 23.68421053  23.68421053 -24.73684211] at step 774
✅ auv1 reached target [ 23.68421053  23.68421053 -24.73684211] at step 775
✅ auv1 reached target [ 23.68421053  23.68421053 -24.73684211] at step 776
✅ auv1 reached target [ 23.68421053  23.68421053 -24.73684211] at step 777
✅ auv2 reached target [ 14.21052632  14.21052632 -19.47368421] at step 919
✅ auv2 reached target [ 14.21052632  14.21052632 -19.47368421] at step 920
✅ auv2 reached target [ 1

KeyboardInterrupt: 